In [ ]:
# Clone the extensions into the web UI's extensions directory
!git clone https://github.com/Mikubill/sd-webui-controlnet.git /content/drive/MyDrive/photo_gen_cloud/sd-webui-controlnet
!git clone https://github.com/huchenlei/sd-webui-openpose-editor.git /content/drive/MyDrive/photo_gen_cloud/sd-webui-openpose-editor

In [ ]:
!git clone https://github.com/Bing-su/adetailer.git /content/drive/MyDrive/photo_gen_cloud/sd-webui-adetailer

Cloning into '/content/drive/MyDrive/photo_gen_cloud/sd-webui-adetailer'...
remote: Enumerating objects: 3075, done.
remote: Counting objects: 100% (970/970), done.
remote: Compressing objects: 100% (121/121), done.
remote: Total 3075 (delta 916), reused 850 (delta 849), pack-reused 2105 (from 3)
Receiving objects: 100% (3075/3075), 562.70 KiB | 6.54 MiB/s, done.
Resolving deltas: 100% (1795/1795), done.


In [ ]:
!pip uninstall mediapipe
!pip install mediapipe

In [ ]:
import mediapipe as mp
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)
print("FaceDetection initialized successfully!")

FaceDetection initialized successfully!


In [ ]:
import torch
from diffusers import EulerAncestralDiscreteScheduler, StableDiffusionControlNetInpaintPipeline, ControlNetModel, DPMSolverMultistepScheduler, StableDiffusionInpaintPipeline
from PIL import Image, ImageFilter
import os
import mediapipe as mp
import numpy as np

# Load the ControlNet model for OpenPose
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/control_v11p_sd15_openpose",
    torch_dtype=torch.float16
)

# Load the model using .bin files
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    controlnet=controlnet,
    torch_dtype=torch.float16,
    use_safetensors=False # Use .bin files instead of safetensors
)
pipe.to("cuda")  # Move to GPU
pipe.enable_attention_slicing()  # Optimize for low VRAM

# Set the scheduler to DPMSolverMultistepScheduler with Karras sigmas (Sampler: DPM++ 2M Karras)
pipe.scheduler = DPMSolverMultistepScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    num_train_timesteps=1000,
    use_karras_sigmas=True
)

#pipe.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe.scheduler.config)

# Function to load images from local paths with error handling
def load_image(path):
    try:
        if not os.path.exists(path):
            raise FileNotFoundError(f"Image not found at {path}")
        image = Image.open(path).convert("RGB").resize((512, 512))
        return image
    except Exception as e:
        print(f"Error loading image: {e}")
        raise

# Local image paths in Colab
init_image_path = "/content/drive/MyDrive/photo_gen_cloud/girl_in_tokyo_street.png"
mask_image_path = "/content/drive/MyDrive/photo_gen_cloud/masked_girl_in_tokyo_street.png"
# pose_image_path =

# Load images
init_image = load_image(init_image_path)
mask_image = load_image(mask_image_path).convert("L")  # Ensure mask is grayscale
# pose_image =

# Apply Gaussian blur to the mask (Mask blur: 0-64)
mask_image = mask_image.filter(ImageFilter.GaussianBlur(radius=32))

# Check mask values (should be 0 and 255 for a proper binary mask)
print(f"Mask min: {mask_image.getextrema()[0]}, max: {mask_image.getextrema()[1]}")

# Define prompts
prompt = "Lady Gaga, highly realistic, facing forward, symmetrical face, detailed hands, delicate fingers, flawless skin, photorealistic portrait, cinematic lighting, sharp details, natural pose"
negative_prompt = "distorted, deformed, extra limbs, bad anatomy, disfigured, uneven eyes, asymmetrical face, blurry, low quality, unnatural proportions, extra fingers, fused hands, artifacts"

# Perform inpainting with settings matching AUTOMATIC1111
results = pipe(
    prompt=prompt,
    negative_prompt=negative_prompt,
    image=init_image,
    mask_image=mask_image,
    #control_image=pose_image,
    strength=0.9,  # Equivalent to Latent Noise for full generation
    guidance_scale=8,  # CFG Scale set to 12
    num_inference_steps=10,  # Increased steps for better quality
    num_images_per_prompt=4  # Batch size of 4
).images

# Face detection and enhancement functions
mp_face_detection = mp.solutions.face_detection

def detect_faces(image_pil, blur_radius=4):
    image_np = np.array(image_pil)
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        results = face_detection.process(image_np)
        masks = []
        if results.detections:
            for detection in results.detections:
                bbox = detection.location_data.relative_bounding_box
                ih, iw, _ = image_np.shape
                xmin = int(bbox.xmin * iw)
                ymin = int(bbox.ymin * ih)
                width = int(bbox.width * iw)
                height = int(bbox.height * ih)
                mask = np.zeros((ih, iw), dtype=np.uint8)
                mask[ymin:ymin+height, xmin:xmin+width] = 255
                mask_pil = Image.fromarray(mask)
                mask_pil = mask_pil.filter(ImageFilter.GaussianBlur(radius=blur_radius))
                masks.append(mask_pil)
        return masks

def enhance_faces(image, pipe, prompt, strength=0.75, guidance_scale=7.5, num_inference_steps=25):
    face_masks = detect_faces(image)
    current_image = image
    for face_mask in face_masks:
        inpainted_image = pipe(
            prompt=prompt,
            image=current_image,
            mask_image=face_mask,
            strength=strength,
            guidance_scale=guidance_scale,
            num_inference_steps=num_inference_steps
        ).images[0]
        current_image = inpainted_image
    return current_image

# Enhance faces in each result and save/display
for i, result in enumerate(results):
    enhanced_image = enhance_faces(result, pipe, prompt)
    enhanced_image.save(f"enhanced_output_image_{i}.png")
    display(enhanced_image)

ModuleNotFoundError: No module named 'mediapipe'